In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q tensorflow-ranking

     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 462 kB 18.4 MB/s 
     |████████████████████████████████| 4.2 MB 8.1 MB/s 
     |████████████████████████████████| 141 kB 7.6 MB/s 


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#movies_f = '/content/drive/MyDrive/Colab Notebooks/Movie-Ratings/movies.csv'
#ratings_f = '/content/drive/MyDrive/Colab Notebooks/Movie-Ratings/ratings.csv'
ratings_f = 'out.csv'

In [ ]:
import pandas as pd
df_out = pd.read_csv("out.csv")
df_out

,Unnamed: 0,userId,movieId,rating,timestamp
0,0,328,2161,3.0,1494212026
1,1,477,1474,2.5,1201158785
2,2,504,6596,3.5,1063644773
3,3,509,5480,1.5,1435996571
4,4,359,736,3.0,1198112116
...,...,...,...,...,...
100831,100831,469,1373,1.0,965334954
100832,100832,599,153236,3.0,1519363838
100833,100833,307,6702,2.5,1186259325
100834,100834,57,3671,4.0,965798574


In [ ]:
test = df_out[80000:]
test

,Unnamed: 0,userId,movieId,rating,timestamp
80000,80000,469,2997,5.0,965333510
80001,80001,583,1777,3.5,1481474338
80002,80002,522,27369,4.0,1254121616
80003,80003,365,89804,2.0,1488594788
80004,80004,432,2018,3.5,1315242295
...,...,...,...,...,...
100831,100831,469,1373,1.0,965334954
100832,100832,599,153236,3.0,1519363838
100833,100833,307,6702,2.5,1186259325
100834,100834,57,3671,4.0,965798574


In [ ]:
# df.sample(frac=1)

In [ ]:
# movies_d = {}

# with open(movies_f, 'r') as f:
#   rows = f.readlines()
#   for row in rows[1:]:
#     row = row.split(",")
#     movies_d[int(row[0])] = row[1]

# print(movies_d)

In [ ]:
import tensorflow as tf

In [ ]:
data = []
ctr = 0
with open(ratings_f, 'r') as f:
  rows = f.readlines()
  for row in rows[1:]:
    row = row.split(",")
    
    r = {}
    
    #out.csv
    r["movie_id"] = tf.convert_to_tensor(row[2])
    r["user_id"] = tf.convert_to_tensor(row[1])
    r["user_rating"] = tf.convert_to_tensor(float(row[3]))

    #ratings.csv
    # r["movie_title"] = tf.convert_to_tensor(movies_d[int(row[1])])
    # r["user_id"] = tf.convert_to_tensor(row[0])
    # r["user_rating"] = tf.convert_to_tensor(float(row[2]))

    data.append(r)

    ctr+=1
    # if ctr ==2:
    #   break

# print(data)
print(len(data))

100836


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data) 

In [ ]:
print(df)

                                            movie_id  \
0         tf.Tensor(b'2161', shape=(), dtype=string)   
1         tf.Tensor(b'1474', shape=(), dtype=string)   
2         tf.Tensor(b'6596', shape=(), dtype=string)   
3         tf.Tensor(b'5480', shape=(), dtype=string)   
4          tf.Tensor(b'736', shape=(), dtype=string)   
...                                              ...   
100831    tf.Tensor(b'1373', shape=(), dtype=string)   
100832  tf.Tensor(b'153236', shape=(), dtype=string)   
100833    tf.Tensor(b'6702', shape=(), dtype=string)   
100834    tf.Tensor(b'3671', shape=(), dtype=string)   
100835     tf.Tensor(b'440', shape=(), dtype=string)   

                                          user_id  \
0       tf.Tensor(b'328', shape=(), dtype=string)   
1       tf.Tensor(b'477', shape=(), dtype=string)   
2       tf.Tensor(b'504', shape=(), dtype=string)   
3       tf.Tensor(b'509', shape=(), dtype=string)   
4       tf.Tensor(b'359', shape=(), dtype=string)   
...      

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(df.to_dict(orient="list"))

In [ ]:
# print(type(ds))

In [ ]:
data_mapped = ds.map(lambda x: {
    "movie_id": x["movie_id"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [ ]:
# print(data_mapped)

In [ ]:
import numpy as np

In [ ]:
tf.random.set_seed(42)
# shuffled = data_mapped.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = data_mapped.take(80_000)
test = data_mapped.skip(80_000).take(20_836)

In [ ]:
movie_titles = data_mapped.batch(1_000_000).map(lambda x: x["movie_id"])
user_ids = data_mapped.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [ ]:
#RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

In [ ]:
import tensorflow_recommenders as tfrs

In [ ]:
from typing import Dict, Text

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError(),
               tf.keras.metrics.MeanAbsoluteError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [ ]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
# cached_train = train.shuffle(100_000).batch(8192).cache()
cached_train = train.batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=300)
# model.fit(train, epochs=30)

Epoch 1/300
10/10 [==============================] - 2s 56ms/step - root_mean_squared_error: 2.0227 - mean_absolute_error: 1.5893 - loss: 3.7731 - regularization_loss: 0.0000e+00 - total_loss: 3.7731
Epoch 2/300
10/10 [==============================] - 0s 42ms/step - root_mean_squared_error: 1.0688 - mean_absolute_error: 0.8531 - loss: 1.1374 - regularization_loss: 0.0000e+00 - total_loss: 1.1374
Epoch 3/300
10/10 [==============================] - 0s 42ms/step - root_mean_squared_error: 1.0189 - mean_absolute_error: 0.8123 - loss: 1.0382 - regularization_loss: 0.0000e+00 - total_loss: 1.0382
Epoch 4/300
10/10 [==============================] - 0s 44ms/step - root_mean_squared_error: 0.9944 - mean_absolute_error: 0.7887 - loss: 0.9893 - regularization_loss: 0.0000e+00 - total_loss: 0.9893
Epoch 5/300
10/10 [==============================] - 0s 40ms/step - root_mean_squared_error: 0.9761 - mean_absolute_error: 0.7711 - loss: 0.9536 - regularization_loss: 0.0000e+00 - total_loss: 0.9536


In [ ]:
model.evaluate(cached_test, return_dict=True)

6/6 [==============================] - 0s 15ms/step - root_mean_squared_error: 0.8832 - mean_absolute_error: 0.6651 - loss: 0.7643 - regularization_loss: 0.0000e+00 - total_loss: 0.7643


{'loss': 0.722289502620697,
 'mean_absolute_error': 0.6651480197906494,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.8831965327262878,
 'total_loss': 0.722289502620697}

In [ ]:
temp_tensor = model({
    "user_id": np.array(["509"]),
    "movie_id": np.array(["736"])
    })

In [ ]:
temp_tensor.numpy()[0][0]

3.039724

In [ ]:
y_pred = {}
y_true = {}
for index, row in test.iterrows():
  y_true[row['userId']] =  y_true.get(row['userId'],[]) + [(row['movieId'],row['rating'])]
  #print(str(int(row['userId'])), str(int(row['movieId'])))
  temp_tensor = model({
    "user_id": np.array([str(int(row['userId']))]),
    "movie_id": np.array([str(int(row['movieId']))])
    })
  val = temp_tensor.numpy()[0][0]
  y_pred[row['userId']] =  y_pred.get(row['userId'],[]) + [(row['movieId'],val)]
print(y_pred)

{469.0: [(2997.0, 4.511436), (1562.0, 1.4460319), (2324.0, 4.037706), (45.0, 3.0977156), (3160.0, 4.1097016), (2412.0, 1.6594483), (2985.0, 3.3430076), (1079.0, 3.915605), (1231.0, 4.002685), (316.0, 3.1041074), (2144.0, 3.828927), (1690.0, 2.6303115), (2288.0, 3.8425522), (89.0, 2.7058723), (3698.0, 2.9728177), (5060.0, 4.2968383), (2393.0, 2.7151685), (2023.0, 3.152612), (39.0, 2.9236083), (10.0, 3.3290567), (1676.0, 3.5176964), (1225.0, 4.3284736), (2550.0, 3.9737031), (1858.0, 2.471403), (3638.0, 2.8436124), (1037.0, 2.7580225), (3178.0, 3.37023), (1754.0, 3.348217), (2470.0, 2.9075763), (3405.0, 4.05445), (2991.0, 3.3244348), (454.0, 3.075324), (260.0, 4.6627326), (1248.0, 4.7646127), (2253.0, 2.1708825), (2202.0, 4.854048), (2300.0, 4.3850756), (1719.0, 3.9308958), (3022.0, 4.3469815), (2203.0, 4.4817076), (1625.0, 3.1048064), (1388.0, 2.113982), (3763.0, 3.341589), (2870.0, 3.6020722), (1247.0, 4.6016154), (2916.0, 3.5696728), (1956.0, 3.7720447), (1304.0, 4.026573), (2331.0, 4.

In [ ]:
for k,v in y_true.items():
  y_true[k].sort(key=lambda x:x[1],reverse=True)

for k,v in y_pred.items():
  y_pred[k].sort(key=lambda x:x[1],reverse=True)

In [ ]:
print(y_pred[1])

[(3147.0, 4.8991985), (1210.0, 4.8358073), (1089.0, 4.8246446), (2502.0, 4.822169), (1206.0, 4.816893), (1258.0, 4.8028073), (1625.0, 4.801135), (6.0, 4.7390246), (3062.0, 4.7250204), (943.0, 4.7051945), (1967.0, 4.7017946), (590.0, 4.683192), (1275.0, 4.6445003), (3671.0, 4.6439204), (1732.0, 4.6208897), (3639.0, 4.5386553), (2947.0, 4.51336), (2058.0, 4.5016212), (1282.0, 4.478723), (70.0, 4.4345493), (101.0, 4.432567), (1097.0, 4.414846), (2033.0, 4.3925376), (2414.0, 4.3627715), (2139.0, 4.3597856), (2406.0, 4.3375325), (4006.0, 4.320827), (3052.0, 4.318343), (1517.0, 4.3029895), (1777.0, 4.3015165), (362.0, 4.294149), (2628.0, 4.2891345), (2090.0, 4.2164664), (2427.0, 4.2056932), (1573.0, 4.189478), (2459.0, 4.1550307), (423.0, 4.1276355), (2141.0, 4.124953), (2093.0, 4.077458), (2644.0, 4.048161), (2048.0, 4.0480857), (1920.0, 4.026797), (1587.0, 3.7290177), (2899.0, 3.4410896)]


In [ ]:
print(y_true[1])

[(2090.0, 5.0), (2502.0, 5.0), (2141.0, 5.0), (1587.0, 5.0), (2947.0, 5.0), (1097.0, 5.0), (362.0, 5.0), (1206.0, 5.0), (2058.0, 5.0), (1625.0, 5.0), (1275.0, 5.0), (3052.0, 5.0), (101.0, 5.0), (1282.0, 5.0), (3671.0, 5.0), (1089.0, 5.0), (2899.0, 5.0), (1517.0, 5.0), (1732.0, 5.0), (3147.0, 5.0), (2048.0, 5.0), (1573.0, 5.0), (2139.0, 5.0), (2033.0, 5.0), (2427.0, 5.0), (1210.0, 5.0), (2459.0, 5.0), (2628.0, 4.0), (1777.0, 4.0), (2644.0, 4.0), (3062.0, 4.0), (590.0, 4.0), (1920.0, 4.0), (2406.0, 4.0), (6.0, 4.0), (943.0, 4.0), (4006.0, 4.0), (1967.0, 4.0), (3639.0, 4.0), (423.0, 3.0), (2414.0, 3.0), (70.0, 3.0), (2093.0, 3.0), (1258.0, 3.0)]


In [ ]:
def recall(y_true, y_pred):
  ctr= 0
  if(len(y_true)<10):
    return 1
  #for i in range(10):
  x = y_pred[:10]
  y = y_true[:10]
  x_list = []
  y_list = []
  for i in range(len(x)):
    x_list.append(x[i][0])
    y_list.append(y[i][0])
  #print(sorted(x_list))
  #print(sorted(y_list))

  for i in y_list:
    if(i in x_list):
      ctr+=1
  return ctr/len(x)

def precision(y_true, y_pred):
  ctr= 0
  if(len(y_true)<10):
    return 1
  #for i in range(10):
  x = y_pred[:10]
  y = y_true[:10]
  x_list = []
  y_list = []
  for i in range(len(x)):
    x_list.append(x[i][0])
    y_list.append(y[i][0])
  #print(sorted(x_list))
  #print(sorted(y_list))

  for i in y_list:
    if(i in x_list):
      ctr+=1
  return ctr/len(y)

def f1_score(precision,recall):
  return 2*precision*recall/(precision+recall)


In [ ]:
def recall_score(y_true, y_pred):
  accumulative_recall = 0
  for i in range(1,610):
    accumulative_recall+=recall(y_true[i], y_pred[i])
  return accumulative_recall/610 

print("Recall: {:.4f}".format(recall_score(y_true, y_pred)))

Recall: 0.6926


In [ ]:
def precision_score(y_true, y_pred):
  accumulative_precision = 0
  for i in range(1,610):
    accumulative_precision+=precision(y_true[i], y_pred[i])
  return accumulative_precision/610

print("Precision: {:.4f}".format(precision_score(y_true, y_pred)))

Precision: 0.6926


In [ ]:
def f1_score(y_true,y_pred):
  p = precision_score(y_true, y_pred)
  r = recall_score(y_true, y_pred)
  return 2*p*r/(p+r)

print("F1-Score: {:.4f}".format(f1_score(y_true, y_pred)))

F1-Score: 0.6926


In [ ]:
from sklearn.metrics import ndcg_score, dcg_score

def nDCG(y_true, y_pred):
  x = y_pred[:10]
  y = y_true[:10]
  x_list = []
  y_list = []
  for i in range(len(x)):
    x_list.append(x[i][0])
    y_list.append(y[i][0])

  return ndcg_score([x_list], [y_list])

In [ ]:
def nDCG_score(y_true, y_pred): 
  accumulative_nDCG = 0
  for i in range(1,610):
    if(len(y_true[i])==1):
      accumulative_nDCG+=1
    else:
      accumulative_nDCG+=nDCG(y_true[i], y_pred[i])
  return accumulative_nDCG/610

print("nDCG-Score: {:.4f}".format(nDCG_score(y_true, y_pred)))

nDCG-Score: 0.7620
